# Credit Risk Prediction under Imbalanced Dataset



In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.utils import resample
from imblearn.under_sampling import ClusterCentroids
from collections import Counter
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
from imblearn.under_sampling import TomekLinks
from collections import Counter
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from utils.opt import evaluate_with_gmean,eval_func, objective_xgb, objective_lgr
from utils.results import ResultsRegistry
from utils.sampling import apply_sampling
import yaml

In [2]:

registry = ResultsRegistry()

In [3]:

with open("utils/config.yaml", "r") as f:
    config = yaml.safe_load(f)

In [4]:

READ_PATH = config["paths_model"]["read_file"]

In [5]:
df = pd.read_csv(READ_PATH)
df

,loan_amnt,funded_amnt_inv,term,installment,annual_inc,delinq_2yrs,open_acc,total_acc,tot_coll_amt,tot_cur_bal,...,income_to_installment,revolver_ratio,total_installment_amt,interest_rate,loan_to_income,loan_to_limit,open_acc_ratio,ever_delinq,emp_length_segment,emp_title_segment
0,16795.0,15094.0,36,32.66,228362.0,0.0,6.0,12.0,5.257495,30634.0,...,582.675036,2552.833333,1175.76,0.077896,0.073546,1.199986,0.500000,False,senior_exp,tech
1,1860.0,1494.0,60,2.17,273685.0,1.0,14.0,20.0,4.110874,158169.0,...,10510.176651,7908.450000,130.20,0.087149,0.006796,0.019137,0.700000,True,mid_exp,tech
2,39158.0,34683.0,36,76.14,165933.0,0.0,11.0,17.0,4.189655,66504.0,...,181.609535,3912.000000,2741.04,0.079031,0.235987,1.237845,0.647059,False,junior_exp,management
3,12284.0,11096.0,60,14.33,136818.0,0.0,7.0,16.0,3.931826,101207.0,...,795.638521,6325.437500,859.80,0.077487,0.089784,0.123025,0.437500,False,senior_exp,management
4,7265.0,6651.0,60,8.48,114755.0,0.0,13.0,15.0,4.488636,108914.0,...,1127.702437,7260.933333,508.80,0.076500,0.063309,0.105605,0.866667,False,junior_exp,professional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14579,9594.0,8471.0,36,18.66,195695.0,0.0,9.0,11.0,5.062595,132947.0,...,873.950518,12086.090909,671.76,0.079301,0.049025,0.189897,0.818182,False,senior_exp,tech
14580,14418.0,12611.0,36,28.04,85130.0,0.0,10.0,16.0,2.484907,99115.0,...,253.001664,6194.687500,1009.44,0.080044,0.169365,0.229729,0.625000,False,senior_exp,tech
14581,35854.0,34497.0,36,69.72,311044.0,0.0,12.0,16.0,3.332205,113572.0,...,371.777587,7098.250000,2509.92,0.072758,0.115270,0.459991,0.750000,False,senior_exp,professional
14582,24108.0,19647.0,36,46.88,288933.0,0.0,9.0,13.0,4.110874,136708.0,...,513.603882,10516.000000,1687.68,0.085900,0.083438,0.971157,0.692308,False,senior_exp,tech


In [6]:
y = df['default']
df.drop(columns=['default'], inplace=True)

categorical_cols = ['term', 'delinq_bucket', 'emp_length_segment', 'emp_title_segment']

df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

In [7]:
X = df.copy()
y = y.astype(int)

## Majorty Undersampling (TomekLinks+ClusterCentroids) and Minorty Oversampling(ADASYN)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)



In [9]:
X_test

,loan_amnt,funded_amnt_inv,installment,annual_inc,delinq_2yrs,open_acc,total_acc,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,...,open_acc_ratio,ever_delinq,term_60,delinq_bucket_1,delinq_bucket_2,emp_length_segment_mid_exp,emp_length_segment_no_exp,emp_length_segment_senior_exp,emp_title_segment_professional,emp_title_segment_tech
14486,5270.0,4708.0,10.25,191180.0,0.0,13.0,18.0,5.945421,31231.0,61385.0,...,0.722222,False,False,False,False,False,False,True,False,False
14058,21547.0,20571.0,25.14,53969.0,0.0,20.0,23.0,4.094345,13674.0,50948.0,...,0.869565,False,True,False,False,True,False,False,False,True
1668,2667.0,2576.0,3.11,311924.0,0.0,8.0,12.0,4.634729,75617.0,104042.0,...,0.666667,False,True,False,False,False,True,False,False,True
9392,30991.0,29086.0,60.26,164472.0,0.0,9.0,15.0,5.056246,40624.0,4219.0,...,0.600000,False,False,False,False,True,False,False,False,False
10474,14546.0,14179.0,28.28,120163.0,0.0,9.0,15.0,5.942799,69072.0,17776.0,...,0.600000,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14373,13303.0,12737.0,25.87,283913.0,0.0,11.0,21.0,3.526361,45828.0,49009.0,...,0.523810,False,False,False,False,False,False,False,True,False
14435,33252.0,31272.0,64.66,287180.0,0.0,4.0,14.0,3.258097,33748.0,55632.0,...,0.285714,False,False,False,False,False,False,False,False,True
1395,6776.0,6243.0,13.18,205632.0,0.0,10.0,16.0,6.729824,160850.0,33173.0,...,0.625000,False,False,False,False,False,True,False,False,True
7426,8642.0,7362.0,16.80,273470.0,1.0,8.0,19.0,6.322565,153622.0,95034.0,...,0.421053,True,False,True,False,False,True,False,False,True


In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

print("Train shape:", X_train.shape, " Test shape:", X_test.shape)
print("Train default oranı:", y_train.mean())
print("Test default oranı:", y_test.mean())

Train shape: (11667, 33)  Test shape: (2917, 33)
Train default oranı: 0.029999142881631954
Test default oranı: 0.030167980802194034


In [11]:
X_final, y_final= apply_sampling(
        X_train, 
        y_train, 
        use_tomek=True,
        use_cluster=True,
        use_random_under=False,
        use_over="adasyn",    
        verbose=True           
    )

📌 Original distribution: Counter({np.int64(0): 11317, np.int64(1): 350})

🔍 Applying TomekLinks...
➡️ After TomekLinks: Counter({np.int64(0): 11176, np.int64(1): 350})

🔻 Applying ClusterCentroids...
➡️ After ClusterCentroids: Counter({np.int64(0): 1050, np.int64(1): 350})

🔺 Applying ADASYN...
➡️ After ADASYN: Counter({np.int64(0): 1050, np.int64(1): 972})

✅ Final distribution: Counter({np.int64(0): 1050, np.int64(1): 972})


## xgboost ile modelleme

In [12]:
X_train, y_train=X_final, y_final

In [13]:
X_train_xbg, X_valid_xgb, y_train_xgb, y_valid_xgb = train_test_split(X_train, y_train, test_size=0.2, random_state=42,stratify=y_train)


In [14]:
neg, pos = np.bincount(y_train)
scale_pos_weight = neg / pos

In [15]:
# Optuna 
study = optuna.create_study(direction="maximize")
study.optimize(lambda trial: objective_xgb(trial,scale_pos_weight,X_train_xbg,X_valid_xgb,y_train_xgb,y_valid_xgb,X), n_trials=20)

print("En iyi hiperparametreler:", study.best_params)
print("En iyi F1 skoru:", study.best_value)
print("En iyi özellikler:", study.best_trial.user_attrs["selected_features"])

[I 2025-11-18 22:50:59,891] A new study created in memory with name: no-name-74ecee15-2c4a-415e-88be-fd685d64bdef
[I 2025-11-18 22:51:01,440] Trial 0 finished with value: 0.8430713082350927 and parameters: {'n_estimators': 231, 'max_depth': 5, 'learning_rate': 0.2704534798842891, 'subsample': 0.8146093835141087, 'colsample_bytree': 0.7263444129482762, 'gamma': 0.8052245467046587, 'min_child_weight': 5, 'reg_alpha': 0.3430889863457087, 'reg_lambda': 2.2252217481289827, 'scale_pos_weight': 0.7869210297072132, 'feature_0': 1, 'feature_1': 0, 'feature_2': 1, 'feature_3': 0, 'feature_4': 1, 'feature_5': 1, 'feature_6': 0, 'feature_7': 0, 'feature_8': 0, 'feature_9': 0, 'feature_10': 1, 'feature_11': 1, 'feature_12': 0, 'feature_13': 0, 'feature_14': 1, 'feature_15': 0, 'feature_16': 1, 'feature_17': 1, 'feature_18': 1, 'feature_19': 1, 'feature_20': 1, 'feature_21': 0, 'feature_22': 1, 'feature_23': 0, 'feature_24': 1, 'feature_25': 1, 'feature_26': 1, 'feature_27': 0, 'feature_28': 1, 'fea

En iyi hiperparametreler: {'n_estimators': 369, 'max_depth': 7, 'learning_rate': 0.2173270364363277, 'subsample': 0.7471536095573567, 'colsample_bytree': 0.883077875794071, 'gamma': 0.10236377035398946, 'min_child_weight': 1, 'reg_alpha': 1.6878809779625268, 'reg_lambda': 6.768828663669709, 'scale_pos_weight': 0.8869359846523067, 'feature_0': 0, 'feature_1': 0, 'feature_2': 0, 'feature_3': 0, 'feature_4': 0, 'feature_5': 1, 'feature_6': 0, 'feature_7': 0, 'feature_8': 1, 'feature_9': 1, 'feature_10': 1, 'feature_11': 0, 'feature_12': 1, 'feature_13': 1, 'feature_14': 1, 'feature_15': 0, 'feature_16': 1, 'feature_17': 1, 'feature_18': 0, 'feature_19': 1, 'feature_20': 1, 'feature_21': 1, 'feature_22': 1, 'feature_23': 0, 'feature_24': 0, 'feature_25': 0, 'feature_26': 0, 'feature_27': 0, 'feature_28': 1, 'feature_29': 0, 'feature_30': 0, 'feature_31': 1, 'feature_32': 0}
En iyi F1 skoru: 0.8729834603817659
En iyi özellikler: ['open_acc', 'tot_cur_bal', 'total_rev_hi_lim', 'emp_length_nu

In [16]:
best_mask = [study.best_trial.params[f"feature_{i}"] for i in range(X.shape[1])]
best_features = [col for col, m in zip(X.columns, best_mask) if m == 1]
print("Seçilen özellikler:", best_features)

Seçilen özellikler: ['open_acc', 'tot_cur_bal', 'total_rev_hi_lim', 'emp_length_num', 'annual_inc_log', 'tot_cur_bal_log', 'utilization_ratio', 'installment_to_income', 'income_to_installment', 'total_installment_amt', 'interest_rate', 'loan_to_income', 'loan_to_limit', 'emp_length_segment_mid_exp', 'emp_title_segment_professional']


In [22]:
feature_names = list(X.columns)

if isinstance(X_train_xbg, np.ndarray):
    X_train_xbg = pd.DataFrame(X_train_xbg, columns=feature_names)

In [23]:
X_train_xbg

,loan_amnt,funded_amnt_inv,installment,annual_inc,delinq_2yrs,open_acc,total_acc,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,...,open_acc_ratio,ever_delinq,term_60,delinq_bucket_1,delinq_bucket_2,emp_length_segment_mid_exp,emp_length_segment_no_exp,emp_length_segment_senior_exp,emp_title_segment_professional,emp_title_segment_tech
0,-0.908363,-0.830146,-0.722356,-1.253376,-0.442640,-1.232049,-1.478787,1.272885,0.520024,1.057499,...,-0.240803,-0.465109,-0.652503,-0.437866,-0.131392,1.322764,-0.327156,-0.612781,1.439684,-1.021575
1,-0.032818,0.113213,0.157307,-1.533137,1.816449,1.227659,0.411778,0.438774,-1.342246,-0.560690,...,1.255656,2.150035,-0.539266,2.283806,-0.131392,-0.648264,-0.327156,1.515576,-0.694597,0.978881
2,0.088437,-0.049868,0.253071,1.167808,-0.442640,-1.008861,-0.604987,0.576218,0.878639,-0.683748,...,-0.980930,-0.465109,-0.395437,-0.437866,-0.131392,-0.755993,-0.327156,-0.612781,-0.694597,0.861207
3,0.760315,0.854590,-0.056860,0.522555,-0.442640,2.972552,2.660848,0.414106,-0.892974,0.285375,...,0.828758,-0.465109,1.532559,-0.437866,-0.131392,0.976305,-0.327156,-0.612781,-0.694597,-0.021347
4,-1.069536,-1.087581,-0.890298,0.115582,4.075537,1.313630,0.505599,0.119526,0.421120,-0.505804,...,1.241815,2.150035,-0.652503,-0.437866,7.610798,1.322764,-0.327156,-0.612781,-0.694597,0.978881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1612,1.138508,1.090337,0.179809,-0.656444,-0.442640,0.428871,-0.011660,-1.223367,0.788573,-0.323411,...,0.772151,-0.465109,1.532559,-0.437866,-0.131392,1.322764,-0.327156,-0.612781,1.155113,-1.021575
1613,0.622148,0.779732,-0.143501,0.161886,-0.442640,0.981845,1.540119,-0.263848,-0.892865,-0.996674,...,-0.220763,-0.465109,1.532559,-0.437866,-0.131392,1.322764,-0.327156,-0.612781,-0.694597,0.978881
1614,0.757331,0.799413,1.015068,-0.603265,1.816449,0.249791,0.288692,0.805679,-0.014751,-0.133676,...,-0.070892,2.150035,-0.652503,2.283806,-0.131392,1.322764,-0.327156,-0.612781,1.439684,-1.021575
1615,1.226462,1.233419,0.234851,-0.191658,-0.442640,0.119206,-0.546162,0.756788,0.519475,-0.824967,...,0.911818,-0.465109,1.532559,-0.437866,-0.131392,1.322764,-0.327156,-0.612781,0.443686,-1.021575


In [24]:
X_train_best = X_train_xbg[best_features]
X_test_best = X_test[best_features]
xgb_best = XGBClassifier(**{k:v for k,v in study.best_params.items() if 'feature_' not in k})
xgb_best.fit(X_train_best, y_train_xgb)

# Threshold optimizasyonu
y_proba_xgb = xgb_best.predict_proba(X_test_best)[:, 1]


best_th_xgb, metrics_xgb, y_pred_xgb = evaluate_with_gmean(y_test, y_proba_xgb)


Best threshold (based on G-Mean): 0.568
Sensitivity: 0.432, Specificity: 0.564, G-Mean: 0.493


In [25]:

y_pred_thresh = (y_proba_xgb >= best_th_xgb).astype(int)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_thresh)
cm

array([[1595, 1234],
       [  50,   38]])

In [26]:
eval_func(y_test, y_pred_thresh)

{'TN': np.int64(1595),
 'FP': np.int64(1234),
 'FN': np.int64(50),
 'TP': np.int64(38),
 'sensitivity': np.float64(0.4318181818181818),
 'specificity': np.float64(0.5638034641215978),
 'g_mean': np.float64(0.4934172542380139)}

In [27]:
registry.add_model(
    model_name="xgb_optuna_gmean",
    best_threshold=best_th_xgb,
    metrics=metrics_xgb,
    y_true=y_test,
    y_pred=y_pred_thresh,
    y_proba=y_proba_xgb,
    model=xgb_best,
    score=metrics_xgb["g_mean"]
)

## Logistic Reg 

In [28]:
X_test_scaled=scaler.transform(X_test)

In [29]:
X_train_scaled = scaler.fit_transform(X_train)


In [30]:
X_train_lgr, X_valid_lgr, y_train_lgr, y_valid_lgr = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42,stratify=y_train)



In [31]:
study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective_lgr(trial,X_train_lgr,y_train_lgr,X_valid_lgr,y_valid_lgr), n_trials=20)


study.optimize(
    lambda trial: objective_lgr(
        trial,
        X_train_lgr,
        y_train_lgr,
        X_valid_lgr,
        y_valid_lgr
    ),
    n_trials=20
)

print("Best trial params:")
print(study.best_trial.params)
print("Best value (G-Mean):", study.best_value)



[I 2025-11-18 22:56:46,351] A new study created in memory with name: no-name-b09c45da-43e7-4b46-a6c1-82436c7a8f87
[I 2025-11-18 22:56:46,399] Trial 0 finished with value: 0.6547282708644164 and parameters: {'C': 0.4636603024597905, 'penalty': 'l2', 'solver': 'lbfgs'}. Best is trial 0 with value: 0.6547282708644164.
c:\conda_envs\kredi_proje\Lib\site-packages\sklearn\linear_model\_logistic.py:1232: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
[I 2025-11-18 22:56:47,080] Trial 1 finished with value: 0.6498238701918099 and parameters: {'C': 0.05806698175015775, 'penalty': 'none', 'solver': 'saga'}. Best is trial 0 with value: 0.6547282708644164.
c:\conda_envs\kredi_proje\Lib\site-packages\sklearn\linear_model\_logistic.py:1232: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
[I 2025-11-18 22:56:47,631] Trial 2 finished with value: 0.6498238701918099 and parameters: {'C': 0.0024928654556067307, 'pe

Best trial params:
{'C': 0.2522532738874124, 'penalty': 'l2', 'solver': 'lbfgs'}
Best value (G-Mean): 0.6595962060739547


In [32]:
best_params = study.best_trial.params

# Modeli oluştur
lr_best = LogisticRegression(
    C=best_params['C'],
    penalty=best_params['penalty'],
    solver=best_params['solver'],
    max_iter=1000,
    class_weight='balanced',
    random_state=42
)
lr_best.fit(X_train_scaled, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,0.2522532738874124
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,42
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [33]:
lr_probs = lr_best.predict_proba(X_test_scaled)[:, 1]


best_th_lgr, metrics_lgr, y_pred_lgr = evaluate_with_gmean(y_test, lr_probs)



Best threshold (based on G-Mean): 0.550
Sensitivity: 0.443, Specificity: 0.604, G-Mean: 0.517


In [34]:
y_pred_thresh_lr = (lr_probs >= best_th_lgr).astype(int)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_thresh_lr)
cm

array([[1709, 1120],
       [  49,   39]])

In [35]:
eval_func(y_test, y_pred_thresh_lr)

{'TN': np.int64(1709),
 'FP': np.int64(1120),
 'FN': np.int64(49),
 'TP': np.int64(39),
 'sensitivity': np.float64(0.4431818181818182),
 'specificity': np.float64(0.6041003888299753),
 'g_mean': np.float64(0.5174227562506425)}

In [36]:
registry.add_model(
    model_name="logreg_optuna_gmean",
    best_threshold=best_th_lgr,
    metrics=metrics_lgr,
    y_true=y_test,
    y_pred=y_pred_thresh_lr,
    y_proba=lr_probs,
    model=lr_best,
    score=metrics_lgr["g_mean"]
)

## Logistic Reg + RandomUnderSampler

In [37]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_resampled, y_resampled= apply_sampling(
        X_train_scaled, 
        y_train, 
        use_tomek=False,
        use_cluster=False,
        use_random_under=True,
        use_over=None,    
        verbose=True           
    )


📌 Original distribution: Counter({np.int64(0): 9901, np.int64(1): 307})

🔻 Applying RandomUnderSampler...
➡️ After RandomUnderSampler: Counter({np.int64(0): 307, np.int64(1): 307})

✅ Final distribution: Counter({np.int64(0): 307, np.int64(1): 307})


In [38]:

X_train_lgr, X_valid_lgr, y_train_lgr, y_valid_lgr = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42,stratify=y_resampled)

# Optuna çalıştır
study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective_lgr(trial,X_train_lgr,y_train_lgr,X_valid_lgr,y_valid_lgr), n_trials=20)

print("Best trial:")
print(study.best_trial.params)

[I 2025-11-18 22:56:51,898] A new study created in memory with name: no-name-0330ea4c-381b-4405-804f-c3f90eeca0c8
c:\conda_envs\kredi_proje\Lib\site-packages\sklearn\linear_model\_logistic.py:1232: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
[I 2025-11-18 22:56:52,214] Trial 0 finished with value: 0.5111139212618532 and parameters: {'C': 0.0069363322052173855, 'penalty': 'none', 'solver': 'saga'}. Best is trial 0 with value: 0.5111139212618532.
c:\conda_envs\kredi_proje\Lib\site-packages\sklearn\linear_model\_logistic.py:1232: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
[I 2025-11-18 22:56:52,257] Trial 1 finished with value: 0.5111139212618532 and parameters: {'C': 0.02372098603422775, 'penalty': 'none', 'solver': 'lbfgs'}. Best is trial 0 with value: 0.5111139212618532.
c:\conda_envs\kredi_proje\Lib\site-packages\sklearn\linear_model\_logistic.py:1232: UserWarning: Setting penalty=None w

Best trial:
{'C': 0.30216224870340613, 'penalty': 'l2', 'solver': 'lbfgs'}


In [39]:
best_params = study.best_trial.params

# Modeli oluştur
lr_best2 = LogisticRegression(
    C=best_params['C'],
    penalty=best_params['penalty'],
    solver=best_params['solver'],
    max_iter=1000,
    class_weight='balanced',
    random_state=42
)
lr_best2.fit(X_train_lgr, y_train_lgr)


lr_probs = lr_best2.predict_proba(X_test_scaled)[:, 1]

# G-Mean'e göre en iyi threshold ve metrikler

best_th_rf, metrics_rf, y_pred_rf = evaluate_with_gmean(y_test, lr_probs)



Best threshold (based on G-Mean): 0.494
Sensitivity: 0.565, Specificity: 0.478, G-Mean: 0.519


In [40]:
y_pred_thresh_rf = (lr_probs >= best_th_rf).astype(int)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_thresh_rf)

In [41]:
registry.add_model(
    model_name="rf_baseline",
    best_threshold=best_th_rf,
    metrics=metrics_rf,
    y_true=y_test,
    y_pred=y_pred_thresh_rf,
    y_proba=lr_probs,
    model=lr_best2,
    score=metrics_rf["g_mean"]
)


In [42]:
confusion_matrix(y_test, y_pred_thresh_rf)

array([[2028, 2217],
       [  57,   74]])

In [43]:
print(registry.results.keys())
for name, res in registry.results.items():
    print("Model:", name)
    print("  threshold:", res.best_threshold)
    print("  metrics:", res.metrics)


dict_keys(['xgb_optuna_gmean', 'logreg_optuna_gmean', 'rf_baseline'])
Model: xgb_optuna_gmean
  threshold: 0.5677063465118408
  metrics: {'tn': np.int64(1595), 'fp': np.int64(1234), 'fn': np.int64(50), 'tp': np.int64(38), 'sensitivity': np.float64(0.4318181818181818), 'specificity': np.float64(0.5638034641215978), 'g_mean': np.float64(0.4934172542380139)}
Model: logreg_optuna_gmean
  threshold: 0.5496544050728125
  metrics: {'tn': np.int64(1709), 'fp': np.int64(1120), 'fn': np.int64(49), 'tp': np.int64(39), 'sensitivity': np.float64(0.4431818181818182), 'specificity': np.float64(0.6041003888299753), 'g_mean': np.float64(0.5174227562506425)}
Model: rf_baseline
  threshold: 0.49409345900449464
  metrics: {'tn': np.int64(2028), 'fp': np.int64(2217), 'fn': np.int64(57), 'tp': np.int64(74), 'sensitivity': np.float64(0.5648854961832062), 'specificity': np.float64(0.4777385159010601), 'g_mean': np.float64(0.5194877848425301)}


In [44]:
run_dir = registry.save_all(base_dir="results", save_models=True)
print(run_dir)

results\run_20251118_225052
